# Reconocimiento de señales de tráfico
### Máster Universitario en Ingeniería computacional y matemática - Área de inteligencia artificial
### Antonio González Hidalgo (agonzalezhidalgo@uoc.edu)

Usando una red neuronal convolucional.

Para la correcta funcionamiento de este notebook, el notebook debe de estar estructurado de la siguiente manera:

- ./dataset
- ./dataset/info.csv          Fichero que contiene los nombres de las señales correspondientes.
- ./dataset/train/Images      Conjunto de imágenes que formarán el entrenamiento de la red neuronal.
- ./dataset/test/info.csv.    Fichero que contiene las categórias de las imágenes de test.
- ./dataset/test/Images       Conjunto de imágenes que constituirán el test.

In [1]:
import import_ipynb
import tfm_generic_functions as traffic

# https://docs.python.org/3/library/os.html
import os

# https://docs.scipy.org/doc/numpy/reference/
import numpy as np

# https://keras.io/models/model/
import keras

from keras import models

# Core Layers: https://keras.io/layers/core/
# Convolution Layers: https://keras.io/layers/convolutional/
from keras import layers

# https://keras.io/preprocessing/image/
from keras.preprocessing.image import ImageDataGenerator

# https://keras.io/callbacks/
from keras.callbacks import ModelCheckpoint

importing Jupyter notebook from tfm_generic_functions.ipynb


Using TensorFlow backend.


In [2]:
# Obtenemos el directorio actual como trabajo.
ROOT_PATH = os.getcwd()

# Establecemos la dimensión de las imágenes.
IMG_SHAPE = (64, 64)

# Configuración de la CNN
ACTIVATION = "elu"
OPTIMIZER = "adam"
EPOCH = 25
BATCH_SIZE = 64

print("Tamaño de las imágenes de entrada: ", IMG_SHAPE)
IMG_SHAPE_LEN = IMG_SHAPE[0] * IMG_SHAPE[1]
print("Vectorizando la entrada, sería de un tamaño: ", IMG_SHAPE_LEN)

# Obtenemos los paths de trabajo
train_path = os.path.join(ROOT_PATH, "dataset_det/train")
test_path = os.path.join(ROOT_PATH, "dataset_det/test")

Tamaño de las imágenes de entrada:  (64, 64)
Vectorizando la entrada, sería de un tamaño:  4096


In [3]:
# Cargamos las imágenes de entrenamiento y de test.
images_train, labels_train = traffic.readDataset(train_path, IMG_SHAPE, False)
images_test, labels_test = traffic.readDataset(test_path, IMG_SHAPE, False)

# Convertimos las listas a array numpy de float32
np_images_train = np.asarray(images_train, dtype = np.float32)
np_labels_train = np.asarray(labels_train, dtype = np.int8)

np_images_test = np.asarray(images_test, dtype = np.float32)
np_labels_test = np.asarray(labels_test, dtype = np.int8)


# Se imprime información de los datos cargados.
traffic.print_size_dataset(images_train, labels_train, np_images_train, np_labels_train, "train")
traffic.print_size_dataset(images_test, labels_test, np_images_test, np_labels_test, "test")

Total images (train):  15481
Total labels (train):  2
Images shape:  (15481, 64, 64, 3)
Labels shape:  (15481,)
Total images (test):  14726
Total labels (test):  2
Images shape:  (14726, 64, 64, 3)
Labels shape:  (14726,)


In [4]:
# Convertimos las labels de manera categórica
labels_categorical_train = keras.utils.to_categorical(np_labels_train)
labels_categorical_test = keras.utils.to_categorical(np_labels_test)

print("Ejemplo primera imagen de manera categórica: ", labels_categorical_train[0])

Ejemplo primera imagen de manera categórica:  [1. 0.]


In [5]:
def get_keras_model(activation):
    # IMPLEMENTACIÓN RED NEURONAL
    # En Keras la envoltura para cualquier red neuronal se crea con la clase Sequential
    model = models.Sequential()

    model.add(layers.Conv2D(32, (5, 5),
                            activation=activation, input_shape=(IMG_SHAPE[0], IMG_SHAPE[1], 3)))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Conv2D(64, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Conv2D(128, (5, 5), activation=activation))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(len(set(labels_train)), activation='softmax'))
    return model

In [6]:
model = get_keras_model(ACTIVATION)

# Muestra la arquitectura de nuestra red neuronal
model.summary()

# Configurando el modelo de aprendijaze:
#  · loss, función para evaluar el grado de error entre salidas calculadas
#  · optimizador, función para calcular los pesos de los parámetros a partir de los datos de entrada
#  · metricas, para monitorizar el proceso de aprendizaje de la red.
model.compile(loss="categorical_crossentropy",
             optimizer=OPTIMIZER,
             metrics=['accuracy'])

# Entrenamiento del modelo
# - batch_size, indica el número de datos que se usan en cada actualización.
# - epochs, indica el número de veces que se usan todos los datos del proceso.
#model.fit(np_images_train, labels_categorical_train, batch_size=32, epochs=20)
model.fit(np_images_train, labels_categorical_train,
          batch_size = BATCH_SIZE,
          epochs = EPOCH,
          verbose = 2,
          callbacks=[ModelCheckpoint('model_64_64_det.h5', save_best_only = False)])

# Evaluación del modelo
test_loss, test_acc = model.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)  

In [7]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False, 
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.2,
                             shear_range=0.1,
                             rotation_range=10.,)

datagen.fit(np_images_test)

In [8]:
model_2 = get_keras_model(ACTIVATION)

model_2.compile(loss="categorical_crossentropy",
             optimizer=OPTIMIZER,
             metrics=['accuracy'])

model_2.fit_generator(datagen.flow(np_images_train, labels_categorical_train, batch_size=32),
                      steps_per_epoch = np_images_train.shape[0],
                      epochs = EPOCH,
                      callbacks=[ModelCheckpoint('model_data_aug_64_64_det.h5',save_best_only = False)])

Epoch 1/25
15481/15481 [==============================] - 3055s 197ms/step - loss: 0.0504 - acc: 0.9872
Epoch 2/25
15481/15481 [==============================] - 3209s 207ms/step - loss: 0.0371 - acc: 0.9936
Epoch 3/25
15481/15481 [==============================] - 2833s 183ms/step - loss: 0.0401 - acc: 0.9946
Epoch 4/25
15481/15481 [==============================] - 2870s 185ms/step - loss: 0.0914 - acc: 0.9931
Epoch 5/25
15481/15481 [==============================] - 3236s 209ms/step - loss: 0.1705 - acc: 0.9891
Epoch 6/25
15481/15481 [==============================] - 2826s 183ms/step - loss: 0.2864 - acc: 0.9821
Epoch 7/25
15481/15481 [==============================] - 2817s 182ms/step - loss: 0.2496 - acc: 0.9845
Epoch 8/25
15481/15481 [==============================] - 2792s 180ms/step - loss: 0.3064 - acc: 0.9809
Epoch 9/25
15481/15481 [==============================] - 2782s 180ms/step - loss: 0.3619 - acc: 0.9775
Epoch 10/25
15481/15481 [==============================] - 2789s

In [9]:
# Evaluación del modelo
test_loss, test_acc = model_2.evaluate(np_images_test, labels_categorical_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

14726/14726 [==============================] - 22s 1ms/step
Test loss: 3.1829024738516156
Test accuracy: 0.8025261442346869
